In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import cv2
import os
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

In [2]:
age_model = tf.keras.models.load_model('age.h5')

In [3]:
gender_model = tf.keras.models.load_model('gender.h5')

In [4]:
race_model = tf.keras.models.load_model('race_detection2_ft_2.h5')

In [42]:
images_dir = "data/"

In [45]:
images = os.listdir(images_dir)
data = []
for file in tqdm(images):
    try:
        image = cv2.imread(images_dir+file,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = faceCascade.detectMultiScale(
                image,
                scaleFactor=1.3,
                minNeighbors=3,
                minSize=(48, 48)
        )
        for (x, y, w, h) in faces:
            face = image[y:y + h, x:x + w]
            data.append(face)
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 408/408 [00:13<00:00, 30.31it/s]


In [46]:
len(data)

114

In [47]:
def age_to_class(age):
    if age<=18:
        return 0
    elif age <= 60:
        return 1
    else:
        return 2

In [48]:
results = []
for face in tqdm(data):
    resized = face/255
    resized = cv2.resize(resized, (48,48) )
    resized = np.expand_dims(resized, axis=0)
    gender = int(np.round(gender_model.predict(resized)))
    age = age_to_class(int(np.round(age_model.predict(resized))))
    resized = cv2.resize(face, (160,160) )
    resized = np.expand_dims(resized, axis=0)
    race = np.argmax(race_model.predict(resized),axis=1)[0]
    results.append({'gender':gender,'age':age,'race':race})

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:13<00:00,  8.22it/s]


In [49]:
df = pd.DataFrame(results)

In [59]:
df = df.reset_index()

In [60]:
df = df.rename(columns={'index':'id'})

In [61]:
df.to_json("chess.json",orient='records') 

In [1]:
import json

In [2]:
with open('chess.json') as f:
    data = json.load(f)

In [4]:
json_str = "var json=" + str(data)

In [26]:
json_str = json_str.replace("'","\"")

In [27]:
with open('report_template.html', 'r') as file:
    html_data = file.read()

In [28]:
k = html_data.replace("var json = []",json_str)

In [19]:
k = ''.join((filter(lambda i: i not in '\\',k)))

In [30]:
with open('report/generated_report.html', 'w') as file:
    file.write(k)